<a href="https://colab.research.google.com/github/RobBurnap/Bioinformatics-MICR4203-MICR5203/blob/main/notebooks/Species_Tree_Diversity_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# BIOINFO4/5203 —
species diveristy:

##A. Mount Google Drive, Import Coding Libraries Necessary for Running Subsequent Code

In [2]:

# Install FIRST, then import
%pip install -q biopython       # Install the Biopython package quietly (-q suppresses most output) so we can work with biological sequence files

from google.colab import drive  # Import the module that lets Colab interact with Google Drive
drive.mount('/content/drive')   # Mount your Google Drive so it appears in Colab's file system under /content/drive

import os, pandas as pd          # Import 'os' for file/directory operations, and pandas for working with data tables
from Bio import SeqIO            # Import SeqIO from Biopython for reading/writing biological sequence files (FASTA, GenBank, etc.)
import matplotlib.pyplot as plt  # Import Matplotlib's plotting library to create figures and graphs

print("✅ Dependencies installed & Drive mounted.")


Mounted at /content/drive
✅ Dependencies installed & Drive mounted.



## B. Course folders: Define the course folders for places to load data to be processed and output to be saved

Edit only `LECTURE_CODE` and `TOPIC` if needed. All inputs will live in `Data/LECTURE_TOPIC` and outputs in `Outputs/LECTURE_TOPIC`.


In [9]:

# --- Course folder config (customize LECTURE_CODE/TOPIC only) ---
COURSE_DIR   = "/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25"
LECTURE_CODE = "L0-species"            # change per week (e.g., L02, L03, ...)
TOPIC        = "diversity"    # short slug for the exercise

# Derived paths (do not change)
DATA_DIR   = f"{COURSE_DIR}/Data/{LECTURE_CODE}_{TOPIC}"
OUTPUT_DIR = f"{COURSE_DIR}/Outputs/{LECTURE_CODE}_{TOPIC}"

# Create folder structure if missing
for p in [f"{COURSE_DIR}/Data", f"{COURSE_DIR}/Outputs", f"{COURSE_DIR}/Notebooks", DATA_DIR, OUTPUT_DIR]:
    os.makedirs(p, exist_ok=True)

print("📁 COURSE_DIR :", COURSE_DIR)
print("📁 DATA_DIR   :", DATA_DIR)
print("📁 OUTPUT_DIR :", OUTPUT_DIR)


📁 COURSE_DIR : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25
📁 DATA_DIR   : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L0-species_diversity
📁 OUTPUT_DIR : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity


##C.



 multi-FASTA of top hits per TaxID (one or more sequences per taxon). The cell below:
	•	uses your existing folders (Data/L0-species_diversity for input; writes to the same folder unless OUTPUT_DIR is already set),
	•	reads query_proteins.fasta and taxids.txt,
	•	for each TaxID, runs a separate BLAST restricted to that taxon (txid####[ORGN]), so we can attribute hits unambiguously,
	•	grabs the top N accessions from each BLAST,
	•	fetches their protein FASTA sequences,
	•	writes:
	•	per_taxid_top_hits.fasta (all sequences, grouped by taxid in headers),
	•	per_taxid_hits.tsv (who came from which taxid, evalue, %id, etc.),
	•	optional one FASTA per taxid (toggle with WRITE_SPLIT_FASTA).

In [ ]:
# --- BLAST (per TaxID) -> collect top protein hits -> write multi-FASTA + TSV ---
from Bio import Entrez, SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from pathlib import Path
import io, csv, re, time, sys

# ==== config you may tweak ====
Entrez.email = "you@university.edu"   # <-- REQUIRED
# Entrez.api_key = "YOUR_NCBI_API_KEY"  # optional, faster/higher limits

TOP_HITS_PER_TAXID = 2        # how many sequences to keep per taxid
EVALUE              = 1e-5
HITLIST_SIZE        = max(50, TOP_HITS_PER_TAXID*10)  # ask for more, then trim
WRITE_SPLIT_FASTA   = False   # also write one FASTA per taxid
SLEEP_BETWEEN_CALLS = 0.3     # be kind to NCBI :)

# ==== paths (use your course vars if present) ====
if 'DATA_DIR' in globals():
    DATA_DIR = Path(DATA_DIR)
else:
    DATA_DIR = Path("/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L0-species_diversity")

if 'OUTPUT_DIR' in globals():
    OUTPUT_DIR = Path(OUTPUT_DIR)
else:
    OUTPUT_DIR = DATA_DIR

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Files we expect from your screenshot:
PROT_FASTA = DATA_DIR / "query_proteins.fasta"
TAXIDS_TXT = DATA_DIR / "taxids.txt"

# ==== load query; auto-detect protein vs nucleotide ====
def load_query():
    if PROT_FASTA.exists():
        rec = next(SeqIO.parse(str(PROT_FASTA), "fasta"))
        seq = str(rec.seq).upper().replace("*","")
        letters = [c for c in seq if c.isalpha()]
        dna_frac = sum(c in set("ACGTN") for c in letters) / max(1,len(letters))
        if dna_frac >= 0.85:
            # looks like nucleotide despite filename; fallback to query.fasta if present
            alt = DATA_DIR / "query.fasta"
            if alt.exists():
                r2 = next(SeqIO.parse(str(alt), "fasta"))
                return ("blastx", str(r2.seq).upper(), f"{alt.name}:{r2.id}")
            # else: we'll still do blastx on given sequence
            return ("blastx", seq, f"{PROT_FASTA.name}:{rec.id}")
        else:
            return ("blastp", seq, f"{PROT_FASTA.name}:{rec.id}")
    else:
        # if protein file missing, try nucleotide and use blastx
        alt = DATA_DIR / "query.fasta"
        if alt.exists():
            r2 = next(SeqIO.parse(str(alt), "fasta"))
            return ("blastx", str(r2.seq).upper(), f"{alt.name}:{r2.id}")
        raise FileNotFoundError(f"Could not find {PROT_FASTA} or a fallback nucleotide FASTA at {DATA_DIR/'query.fasta'}")

mode, query_seq, query_label = load_query()
print(f"📄 Query source: {query_label}")
print(f"🧪 Mode chosen: {mode.upper()} vs nr  | length={len(query_seq)}")

# ==== load TaxIDs ====
if not TAXIDS_TXT.exists():
    raise FileNotFoundError(f"taxids.txt not found at {TAXIDS_TXT}")
taxids = [t.strip() for t in TAXIDS_TXT.read_text().splitlines() if t.strip().isdigit()]
if not taxids:
    raise ValueError("taxids.txt is empty or contains no numeric TaxIDs.")
print(f"🧬 Loaded {len(taxids)} TaxIDs")

# ==== helpers ====
def run_single_blast(seq, taxid, program="blastp", evalue=EVALUE, hitlist=HITLIST_SIZE):
    q = f"txid{taxid}[ORGN]"  # exact taxon; robust for BLAST
    prev = q if len(q) < 160 else (q[:157] + " …")
    print(f"⏳ {program.upper()} vs nr | taxid={taxid} | E={evalue} | hits={hitlist}\n   ENTREZ_QUERY: {prev}")
    h = NCBIWWW.qblast(program=program, database="nr", sequence=seq,
                       expect=evalue, entrez_query=q,
                       hitlist_size=hitlist, descriptions=hitlist, alignments=hitlist)
    xml = h.read(); h.close()
    rec = NCBIXML.read(io.StringIO(xml))
    return rec, xml

def extract_accession(hit_id, hit_def, accession_attr):
    if accession_attr: return accession_attr
    m = re.search(r"([A-Z]{1,3}_?\d+\.\d+)", hit_id or "")
    if m: return m.group(1)
    m = re.search(r"([A-Z]{1,3}_?\d+\.\d+)", hit_def or "")
    if m: return m.group(1)
    return (hit_id or "unknown")

def fetch_protein_fasta(accessions):
    """Return dict acc->FASTA text (single-record each)."""
    out = {}
    batch = list(accessions)
    while batch:
        # fetch in small batches to be polite
        chunk = batch[:50]; batch = batch[50:]
        try:
            h = Entrez.efetch(db="protein", id=",".join(chunk), rettype="fasta", retmode="text")
            txt = h.read(); h.close()
            # split on records
            parts = [t for t in txt.strip().split(">") if t]
            for rec_txt in parts:
                header, *seq_lines = rec_txt.splitlines()
                acc = header.split()[0]
                out[acc] = ">" + header + "\n" + "\n".join(seq_lines) + "\n"
        except Exception as e:
            sys.stderr.write(f"[warn] efetch failed for {chunk}: {e}\n")
        time.sleep(SLEEP_BETWEEN_CALLS)
    return out

# ==== main loop: BLAST per TaxID -> keep top N -> fetch FASTA ====
all_rows = []
per_taxid_fastas = {}   # taxid -> list of FASTA strings
xml_paths = []

for i, tid in enumerate(taxids, 1):
    try:
        record, xml = run_single_blast(query_seq, tid, program=("blastp" if mode=="blastp" else "blastx"))
        # save each XML (handy for grading/debug)
        xml_file = OUTPUT_DIR / f"{mode}_nr_taxid{tid}.xml"
        xml_file.write_text(xml); xml_paths.append(xml_file)

        if not record.alignments:
            print(f"— No hits for taxid {tid}")
            continue

        # pick top-N by (evalue, -identities)
        hsps = []
        for aln in record.alignments:
            best = sorted(aln.hsps, key=lambda h: (h.expect, -h.identities))[0]
            acc = extract_accession(aln.hit_id, aln.hit_def, getattr(aln, "accession", None))
            pct_id = 100.0 * best.identities / best.align_length if best.align_length else 0.0
            hsps.append((aln, best, acc, pct_id))
        hsps.sort(key=lambda t: (t[1].expect, -t[3]))
        keep = hsps[:TOP_HITS_PER_TAXID]

        # fetch FASTAs for these accessions
        accs = [acc for _,_,acc,_ in keep]
        acc_to_fa = fetch_protein_fasta(accs)

        # store
        per_taxid_fastas.setdefault(tid, [])
        for aln, best, acc, pct in keep:
            fa = acc_to_fa.get(acc)
            if not fa:
                continue
            # prepend taxid info to header for grouping
            # keep original header after a pipe
            lines = fa.strip().splitlines()
            header = lines[0][1:]  # drop '>'
            seq = "\n".join(lines[1:])
            new_header = f">taxid:{tid}|acc:{acc}|e:{best.expect:.2e}|pid:{pct:.2f}|len:{best.align_length} {header}"
            per_taxid_fastas[tid].append(new_header + "\n" + seq + "\n")

            all_rows.append([
                tid, acc, aln.title, aln.length, best.expect,
                best.identities, best.align_length, round(pct,2),
                min(best.query_start,best.query_end), max(best.query_start,best.query_end),
                min(best.sbjct_start,best.sbjct_end), max(best.sbjct_start,best.sbjct_end)
            ])
        print(f"✅ taxid {tid}: kept {len(per_taxid_fastas[tid])} sequences")
    except Exception as e:
        print(f"⚠️ taxid {tid} failed: {e}")
    time.sleep(SLEEP_BETWEEN_CALLS)

# ==== write combined multi-FASTA + per-taxid FASTAs ====
multi_fa = OUTPUT_DIR / "per_taxid_top_hits.fasta"
with open(multi_fa, "w") as fh:
    for tid in taxids:
        for fa in per_taxid_fastas.get(tid, []):
            fh.write(fa)
print(f"💾 Multi-FASTA: {multi_fa}")

if WRITE_SPLIT_FASTA:
    for tid, fas in per_taxid_fastas.items():
        p = OUTPUT_DIR / f"taxid_{tid}_top_hits.fasta"
        with open(p, "w") as fh:
            for fa in fas: fh.write(fa)
    print("💾 Also wrote per-taxid FASTAs")

# ==== write table ====
tsv = OUTPUT_DIR / "per_taxid_hits.tsv"
with open(tsv, "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["taxid","accession","title","subject_length","evalue","identities","align_len","pct_identity","q_start","q_end","s_start","s_end"])
    w.writerows(all_rows)
print(f"📑 Table: {tsv}")

print(f"🗂 XML files saved: {len(xml_paths)}")

📄 Query source: query_proteins.fasta:unknown_seq
🧪 Mode chosen: BLASTP vs nr  | length=475
🧬 Loaded 30 TaxIDs
⏳ BLASTP vs nr | taxid=10090 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid10090[ORGN]
✅ taxid 10090: kept 0 sequences
⏳ BLASTP vs nr | taxid=10116 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid10116[ORGN]
✅ taxid 10116: kept 0 sequences
⏳ BLASTP vs nr | taxid=102285 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid102285[ORGN]
✅ taxid 102285: kept 0 sequences
⏳ BLASTP vs nr | taxid=105358 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid105358[ORGN]
✅ taxid 105358: kept 0 sequences
⏳ BLASTP vs nr | taxid=1148 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1148[ORGN]
✅ taxid 1148: kept 0 sequences
⏳ BLASTP vs nr | taxid=1280 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1280[ORGN]
✅ taxid 1280: kept 0 sequences
⏳ BLASTP vs nr | taxid=1299 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1299[ORGN]
✅ taxid 1299: kept 0 sequences
⏳ BLASTP vs nr | taxid=1423 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid1423[ORGN]
✅ taxid 1423: k

/usr/local/lib/python3.11/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request A7439PK2015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


✅ taxid 2173: kept 0 sequences
⏳ BLASTP vs nr | taxid=2190 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2190[ORGN]


/usr/local/lib/python3.11/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request A74SJWZK015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


✅ taxid 2190: kept 0 sequences
⏳ BLASTP vs nr | taxid=2234 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2234[ORGN]


/usr/local/lib/python3.11/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request A75BY7M0015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


✅ taxid 2234: kept 0 sequences
⏳ BLASTP vs nr | taxid=2242 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2242[ORGN]
✅ taxid 2242: kept 0 sequences
⏳ BLASTP vs nr | taxid=2246 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2246[ORGN]
✅ taxid 2246: kept 0 sequences
⏳ BLASTP vs nr | taxid=2287 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2287[ORGN]
✅ taxid 2287: kept 0 sequences
⏳ BLASTP vs nr | taxid=2303 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid2303[ORGN]
✅ taxid 2303: kept 0 sequences
⏳ BLASTP vs nr | taxid=271 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid271[ORGN]


/usr/local/lib/python3.11/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request A773KGHC015 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


✅ taxid 271: kept 0 sequences
⏳ BLASTP vs nr | taxid=287 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid287[ORGN]
✅ taxid 287: kept 0 sequences
⏳ BLASTP vs nr | taxid=338192 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid338192[ORGN]
✅ taxid 338192: kept 0 sequences
⏳ BLASTP vs nr | taxid=562 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid562[ORGN]
✅ taxid 562: kept 0 sequences
⏳ BLASTP vs nr | taxid=63221 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid63221[ORGN]
— No hits for taxid 63221
⏳ BLASTP vs nr | taxid=666 | E=1e-05 | hits=50
   ENTREZ_QUERY: txid666[ORGN]


/usr/local/lib/python3.11/dist-packages/Bio/Blast/NCBIWWW.py:275: BiopythonWarning: BLAST request A78C5S8T013 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(


In [20]:
from Bio.Blast import NCBIXML
from Bio import Entrez
from pathlib import Path
import re, time

# REQUIRED
Entrez.email = "your_email@university.edu"   # <-- change me
# Entrez.api_key = "YOUR_NCBI_API_KEY"       # optional but helpful

# Paths (match your screenshots)
# DATA_DIR  = Path("/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Data/L0-species_diversity")
# OUT_FASTA = DATA_DIR / "blast_hits.fasta"

# Use the OUTPUT_DIR defined in the previous cell
if 'OUTPUT_DIR' in globals():
    OUTPUT_DIR = Path(OUTPUT_DIR)
else:
    # Fallback if the variable is not set
    OUTPUT_DIR = Path("/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity")

OUT_FASTA = OUTPUT_DIR / "blast_hits.fasta"


# Find XML files (your files look like 'blastp_nr_taxid*.xml')
# XMLS = sorted(DATA_DIR.glob("blastp_nr_taxid*.xml"))
# if not XMLS:
#     XMLS = sorted(DATA_DIR.glob("*.xml"))
# assert XMLS, f"No XML files found in {DATA_DIR}"

# Look for XML files in the OUTPUT_DIR
XMLS = sorted(OUTPUT_DIR.glob("blastp_nr_taxid*.xml"))
if not XMLS:
    XMLS = sorted(OUTPUT_DIR.glob("*.xml")) # Fallback to any xml if specific not found

assert XMLS, f"No XML files found in {OUTPUT_DIR}"


MAX_HITS_PER_XML = 5     # top N alignments per XML
BATCH = 50               # efetch batch size
SLEEP = 0.25             # courtesy to NCBI

def parse_top_accessions(xml_path, top_n=5):
    """Return list of accession strings from top alignments in one BLAST XML."""
    with open(xml_path) as h:
        rec = NCBIXML.read(h)
    accs = []
    for aln in rec.alignments[:top_n]:
        # Prefer parser's accession if available; else regex from id/def/title
        acc = getattr(aln, "accession", None)
        if not acc:
            for field in (aln.hit_id, aln.hit_def, aln.title):
                m = re.search(r"([A-Z]{1,5}_?\d+(?:\.\d+)?)", field or "")
                if m:
                    acc = m.group(1); break
        if acc:
            accs.append(acc)
    return accs

def normalize_pair(acc):
    """Return (base, versioned) forms so XOB97566 and XOB97566.1 both match."""
    base = acc.split(".")[0]
    ver  = acc if "." in acc else None
    return base, ver

def fetch_fasta_batched(accessions):
    """Fetch FASTA for a list of accessions; store under both base & version keys."""
    out = {}
    accs = [a for a in set(accessions) if a]
    for i in range(0, len(accs), BATCH):
        chunk = accs[i:i+BATCH]
        try:
            handle = Entrez.efetch(db="protein", id=",".join(chunk), rettype="fasta", retmode="text")
            text = handle.read(); handle.close()
            for block in [b for b in text.strip().split(">") if b]:
                header, *seq_lines = block.splitlines()
                header = header.strip()
                seq = "\n".join(seq_lines).strip()
                token = header.split()[0]        # e.g., XOB97566.1
                base = token.split(".")[0]
                fasta = f">{header}\n{seq}\n"
                out[token] = fasta
                out[base]  = fasta
        except Exception as e:
            print(f"[warn] efetch failed for {chunk}: {e}")
        time.sleep(SLEEP)
    return out

# --- Collect top accessions from all XMLs
wanted = []
for x in XMLS:
    accs = parse_top_accessions(x, MAX_HITS_PER_XML)
    if accs:
        wanted.extend(accs)
    else:
        print(f"— {x.name}: no alignments or no accessions parsed")

# Also include versionless forms to improve retrieval
expanded = []
for a in set(wanted):
    base, ver = normalize_pair(a)
    expanded.append(base)
    if ver: expanded.append(ver)

# --- Fetch FASTA
acc_to_fasta = fetch_fasta_batched(expanded)

# --- Write combined FASTA (keep original headers; add source xml in a comment line)
written = 0
with open(OUT_FASTA, "w") as out:
    for x in XMLS:
        accs = parse_top_accessions(x, MAX_HITS_PER_XML)
        for acc in accs:
            # try versioned, then base
            fa = acc_to_fasta.get(acc) or acc_to_fasta.get(acc.split(".")[0])
            if not fa:
                print(f"[miss] No FASTA for {acc} (from {x.name})")
                continue
            # insert a comment line indicating source XML
            header, *seq_lines = fa.strip().splitlines()
            out.write(header + f"  ; source={x.name}\n")
            out.write("\n".join(seq_lines) + "\n")
            written += 1

print(f"💾 Wrote {written} sequences to {OUT_FASTA}")

💾 Wrote 12 sequences to /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity/blast_hits.fasta


In [17]:
# --- Augment your species list with mammals + bacteria + archaea, resolve -> TaxIDs, and merge with existing ---
from Bio import Entrez
from pathlib import Path
import pandas as pd, re, time, sys

Entrez.email = "you@university.edu"   # <-- set your email

assert 'OUTPUT_DIR' in globals() and 'DATA_DIR' in globals()
OUT = Path(OUTPUT_DIR); OUT.mkdir(parents=True, exist_ok=True)

ADD_THESE_NAMES = [
    # Mammals
    "Mus musculus", "Rattus norvegicus", "Bos taurus", "Tursiops truncatus",
    "Gorilla gorilla", "Pan troglodytes", "Pan paniscus", "Pongo abelii",
    "Homo neanderthalensis", "Homo sapiens",
    # Bacteria
    "Escherichia coli", "Bacillus subtilis", "Staphylococcus aureus",
    "Pseudomonas aeruginosa", "Mycobacterium tuberculosis",
    "Streptomyces coelicolor", "Deinococcus radiodurans",
    "Thermus aquaticus", "Synechocystis sp. PCC 6803", "Vibrio cholerae",
    # Archaea
    "Methanocaldococcus jannaschii", "Haloferax volcanii", "Halobacterium salinarum",
    "Sulfolobus solfataricus", "Archaeoglobus fulgidus", "Thermoplasma acidophilum",
    "Nitrosopumilus maritimus", "Methanobrevibacter smithii",
]

def tax_lookup(name: str):
    """Resolve scientific name -> (taxid, canonical, rank). Try strict then loose."""
    queries = [f"\"{name}\"[SCIN]", name]
    tried = set()
    for q in queries:
        if q in tried:
            continue
        tried.add(q)
        try:
            h = Entrez.esearch(db="taxonomy", term=q, retmode="xml"); r = Entrez.read(h); h.close()
            if int(r["Count"]) == 0:
                continue
            tid = r["IdList"][0]
            h2 = Entrez.efetch(db="taxonomy", id=tid, retmode="xml"); rec = Entrez.read(h2); h2.close()
            node = rec[0]
            return int(tid), node.get("ScientificName",""), node.get("Rank","")
        except Exception as e:
            sys.stderr.write(f"[warn] {name}: {e}\n")
            continue
    return "", "", ""

rows = []
seen = set()
for nm in ADD_THESE_NAMES:
    nm = nm.strip()
    if not nm or nm in seen:
        continue
    seen.add(nm)
    tid, canon, rank = tax_lookup(nm)
    rows.append({"input_name": nm, "taxid": tid, "canonical_name": canon, "rank": rank})
    time.sleep(0.25)

df_new = pd.DataFrame(rows)
map_path = OUT / "augmented_taxon_map.csv"
df_new.to_csv(map_path, index=False)

# Load existing taxids (if present) and merge
existing_taxids = set()
for p in [OUT/"taxids.txt", OUT/"taxids-augmented.txt", Path(DATA_DIR)/"taxids.txt"]:
    if p.exists():
        existing_taxids |= {t.strip() for t in p.read_text().splitlines() if t.strip().isdigit()}

new_taxids = {str(t) for t in df_new["taxid"].tolist() if str(t).isdigit()}
all_taxids = sorted(existing_taxids | new_taxids)

# Write merged lists
(OUT / "taxids-augmented.txt").write_text("\n".join(all_taxids))

# Build inclusive Entrez query (descendants by default; safer for genus/“sp.”)
eq_exp = " OR ".join([f"txid{t}[ORGN]" for t in all_taxids])  # use [ORGN] for BLAST robustness
(OUT / "entrez_query_exp_aug.txt").write_text(eq_exp)

print(f"✅ Resolved {len(new_taxids)} new taxa. Total now: {len(all_taxids)}")
print("🗺  Map CSV :", map_path)
print("🧬 TaxIDs  :", OUT / 'taxids-augmented.txt')
print("🔎 ENTREQ  :", OUT / 'entrez_query_exp_aug.txt')

✅ Resolved 28 new taxa. Total now: 30
🗺  Map CSV : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity/augmented_taxon_map.csv
🧬 TaxIDs  : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity/taxids-augmented.txt
🔎 ENTREQ  : /content/drive/MyDrive/Teaching/BIOINFO4-5203-F25/Outputs/L0-species_diversity/entrez_query_exp_aug.txt
